## Initial textual processing
In this file we process and extract key data from the batch for later analysis.

* Load batch
* Parse keystrokes
* Extract if its pasted

In [95]:
import pandas as pd 

In [103]:
df = pd.read_csv("../data/Batch_5083843_batch_results.csv")
# df_new = pd.read_csv("../data/Batch_newest.csv")

In [106]:
def parse_response(x):
    """
    Create proper datastructure for futher anaylsis
    """
    summary = x["summary"]
    actions = eval(x["log_of_what_they_did"])
    
    data = {"keys": "", "dates": []}
    
    for val in actions:
        if (val[0] == "copy") or (val[0] == "paste"):
            data["keys"] += " " + val[0]
            data["dates"].append(None)
        else:
            data["keys"] += " " + val[1]
            data["dates"].append(val[2])
    data["keys"] = " ".join(data["keys"].split()).strip()
    
    return summary, data

In [100]:
# print(df["Answer.taskAnswers"].iloc[5])

In [110]:
df.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.texts',
       'Answer.taskAnswers', 'Approve', 'Reject'],
      dtype='object')

In [111]:
relevant_columns = ["HITId", "original_text","key_strokes", "datetime", "summary", "WorkerId"]

responses = pd.DataFrame(columns = relevant_columns)

for i, row in df.iterrows():
    unparsed = eval(row["Answer.taskAnswers"])[0]
    summary, data = parse_response(unparsed)
    
    key_strokes = data["keys"]
    datetime = data["dates"]
    worker_id = row["WorkerId"]
    
    temp = pd.DataFrame([[row["HITId"], row["Input.texts"], key_strokes, datetime, summary, worker_id]], columns = relevant_columns)
    
    responses = pd.concat([responses, temp], axis = 0)
    
    

In [87]:
# responses

In [112]:
def locate_paste(x: str) -> bool:
    """
    This function will attempt to use some heuristics to determine
    if the key strokes signify that the text was copied.
    """
    if ("Control v" in x) or ("paste" in x):
        return True
    else:
        return False
    
responses["copied"] = responses["key_strokes"].apply(lambda x: locate_paste(x))

In [113]:
responses.groupby("copied")["HITId"].count()

copied
False     5
True     43
Name: HITId, dtype: int64

In [90]:
# responses = responses.drop_duplicates(subset="summary")

In [114]:
print(len(responses))
reponses = responses.drop_duplicates(subset=["summary"])
print(len(responses))

48
48


In [115]:
responses

,HITId,original_text,key_strokes,datetime,summary,WorkerId,copied
0,309D674SHZ6NLN8CVPB7DNTRX7EBCC,Comparison of Weight-Loss Diets with Different...,Backspace Backspace Backspace Backspace Backsp...,"[2023-06-01T07:06:03.732Z, 2023-06-01T07:06:03...",We randomly assigned 811 overweight adults to ...,A2HFHW1AT6CYCV,True
0,309D674SHZ6NLN8CVPB7DNTRX7EBCC,Comparison of Weight-Loss Diets with Different...,copy paste copy paste,"[None, None, None, None]","onths, participants assigned to each diet had ...",A7ET4QBOZ9AEV,True
0,309D674SHZ6NLN8CVPB7DNTRX7EBCC,Comparison of Weight-Loss Diets with Different...,copy paste,"[None, None]",In this two-year study of 811 overweight adult...,AG6AHUZK4KU3,True
0,3XABXM4AJ1Q8DQKOXVSS6QS7TGU8QF,Effect of Screening Mammography on Breast-Canc...,Shift Shift Shift Shift Shift Shift Shift Shif...,"[2023-06-01T10:42:22.995Z, 2023-06-01T10:42:23...",The Norwegian breast-cancer screening program ...,A2NZK4Z73D4FQB,False
0,3XABXM4AJ1Q8DQKOXVSS6QS7TGU8QF,Effect of Screening Mammography on Breast-Canc...,copy Control V paste,"[None, 2023-06-01T06:39:50.070Z, 2023-06-01T06...",This study examined the effect of screening ma...,A2TJ8LWLUO31Q,True
0,3XABXM4AJ1Q8DQKOXVSS6QS7TGU8QF,Effect of Screening Mammography on Breast-Canc...,copy paste Control v paste,"[None, None, 2023-06-01T06:40:06.904Z, 2023-06...",The effect of screening mammography on breast ...,A3A31C77CXQ7U9,True
0,3BFF0DJK8XX3OQRPSXSSSM3VPHZTSD,"First Results of Phase 3 Trial of RTS,S/AS01 M...",copy copy CapsLock Shift Control V paste,"[None, None, 2023-06-01T16:28:11.659Z, 2023-06...",Vaccine efficacy against severe malaria in th...,A3NMWNY6DTELXW,True
0,3BFF0DJK8XX3OQRPSXSSSM3VPHZTSD,"First Results of Phase 3 Trial of RTS,S/AS01 M...",copy Control v paste,"[None, 2023-06-01T06:44:19.371Z, 2023-06-01T06...",The first results of a phase 3 trial for the R...,A3VDG4N48AVYGQ,True
0,3BFF0DJK8XX3OQRPSXSSSM3VPHZTSD,"First Results of Phase 3 Trial of RTS,S/AS01 M...",copy paste,"[None, None]",\nIn the 14 months after the first dose of vac...,A96HZQTUTFWL6,True
0,3CESM1J3EIOQT9XMGPU0E1E2BMKW6K,Azithromycin and the Risk of Cardiovascular De...,copy Control V paste,"[None, 2023-06-01T06:39:08.321Z, 2023-06-01T06...","In this study, a Tennessee Medicaid cohort was...",A1BOLZ1ZMZGUJ9,True


In [92]:
responses.to_csv("../data/processed_responses.csv", index=False)